In [1]:
#!pip install SPARQLWrapper
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install seaborn
#!pip install sparqlwrapper
#!pip install plotly

In [2]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from SPARQLWrapper import SPARQLWrapper, N3, JSON
from rdflib import Graph
import plotly.graph_objects as go

import plotly.express as px
import re



%matplotlib inline

In [3]:
#endpoint = "http://localhost:9999/blazegraph/sparql" # SPARQL endpoint hosting ITO.owl

endpoint = "http://149.148.106.153:9999/blazegraph/sparql"
prefixes = """
PREFIX edam: <http://edamontology.org/>
PREFIX obo:  <http://purl.obolibrary.org/obo/>
PREFIX ito:  <https://identifiers.org/ito#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
"""

def query(service, query, numeric_cols = []):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
        
    df = pd.DataFrame(out, columns=cols)
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
    
    return df


In [4]:
#First select here the metricName that we will use

#desired_measure = "Accuracy"
#desired_measure = "Hits-at-10"
#desired_measure = "Hits-at-1"
#desired_measure = "MRR"
#desired_measure = "AUC"

query = """
    PREFIX edam: <http://edamontology.org/>
    PREFIX obo:  <http://purl.obolibrary.org/obo/>
    PREFIX ito:  <https://identifiers.org/ito:>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT *
    WHERE {
        ?paper a edam:data_0971 . 
        ?paper rdfs:label ?paperTitle. 
        ?paper obo:date ?date. 

        ?benchmark_process_individual 	rdfs:seeAlso ?paper ;
                                        rdfs:label ?model_label ;
                                        a ?benchmark_process_class . # this will create a place holder for the rdfs:type results that contains the information about the individual
        
        ?benchmark_process_class rdfs:label ?benchmark_process_class_label ;
                                 rdfs:subClassOf* ito:ITO_00141 . # this limits for the NLP class


        ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
        ?performance_measure rdfs:label ?metricName .



        ?performance_measure rdfs:label ?metricName .
        ?benchmark_process_individual ?performance_measure ?result .	


        #FILTER regex(?paperTitle, "Quaternion Knowledge Graph Embeddings" ). # this searches a specific match
        #FILTER regex(?benchmark_process_class_label, "- Link Prediction benchmarking" ). # this searches a specific match
        
        FILTER regex(?benchmark_process_class_label, " - Sentiment Analysis benchmarking" ). # this searches a specific match

        #WN18RR - Link Prediction benchmarking




    } ORDER by ?date



    """

#send query via API
sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
result = sparql.query()

#process results as JSON
processed_results = json.load(result.response)

#Use accessory function to process results
cols = processed_results['head']['vars']

out = []
for row in processed_results['results']['bindings']:
    item = []
    for c in cols:
        item.append(row.get(c, {}).get('value'))

        out.append(item)


#this is the final df containing the results of the query
df2 = pd.DataFrame(out, columns=cols)

#get unique benchmark_process_class_label
#benchmark_process_class_label = df2["benchmark_process_class_label"].unique()

#get unique metricName
metricName = df2["metricName"].unique()


print("Number of metrics: ",len(metricName))

for desired_measure in metricName:
    print ("|"+desired_measure+"|")

Number of metrics:  14
|1 in 10 R-at-1|
|Accuracy|
|Average|
|DVD|
|Books|
|Electronics|
|Kitchen|
|Error|
|F1-score|
|F1|
|Average Recall|
|MSE|
|R^2|
|F1 score|


In [5]:
#Query database and get one ratio dataframe per mesure. Example provided.


def create_ratio(desired_measure, desired_benchmark, ds_count):
    print("Creating ratio df for ",desired_measure,", ",desired_benchmark,", ds_count=",ds_count)
    query = """
    PREFIX edam: <http://edamontology.org/>
    PREFIX obo:  <http://purl.obolibrary.org/obo/>
    PREFIX ito:  <https://identifiers.org/ito:>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT *
    WHERE {
        ?paper a edam:data_0971 . 
        ?paper rdfs:label ?paperTitle. 
        ?paper obo:date ?date. 

        ?benchmark_process_individual 	rdfs:seeAlso ?paper ;
                                        rdfs:label ?model_label ;
                                        a ?benchmark_process_class . # this will create a place holder for the rdfs:type results that contains the information about the individual
        
        ?benchmark_process_class rdfs:label ?benchmark_process_class_label ;
                                 rdfs:subClassOf* ito:ITO_00141 . # this limits for the NLP class
        

        ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
        ?performance_measure rdfs:label ?metricName .



        ?performance_measure rdfs:label ?metricName .
        ?benchmark_process_individual ?performance_measure ?result .	


        #FILTER regex(?paperTitle, "Quaternion Knowledge Graph Embeddings" ). # this searches a specific match
        FILTER regex(?metricName, "^"""+ desired_measure +"""$" ). # this searches a specific match
        FILTER regex(?benchmark_process_class_label, "^"""+desired_benchmark+"""$" ). # this searches a specific match
        #FILTER regex(?benchmark_process_class_label, "- Link Prediction benchmarking" ). # this searches a specific match
        FILTER regex(?benchmark_process_class_label, " - Sentiment Analysis benchmarking" ). # this searches a specific match
        #WN18RR - Link Prediction benchmarking




    } ORDER by ?date



    """

    #send query via API
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    #process results as JSON
    processed_results = json.load(result.response)

    #Use accessory function to process results
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))

        out.append(item)

    #this is the final df containing the results of the query
    df = pd.DataFrame(out, columns=cols)
    
    if len(df.index)==0:
        return None
           
    #Set date column to datetime
    df['date'] = pd.to_datetime(df['date'])

    #Keep only the model name
    df.new = df.replace(to_replace =' model in .*$', value = '', regex = True)

    df.new["result"] = pd.to_numeric(df.new["result"]) 

    #Create dataframe to store state of art results
    sota = pd.DataFrame(columns = df.new.columns)

    best_value = 0
    i=0

    for result in df.new["result"]:        
            if result > best_value:
                best_value = result
                sota_add = pd.DataFrame(df.new.iloc[i]).transpose()
                sota = sota.append(sota_add)            
            i = i+1    

    #This variable is containing the data points to plot the state of the art line        
    #sota

    #this is a copy to use as hovertemplate below
    y=df.new["result"].astype(str)


    #create this data frame to store the ratios
    ratio_df = pd.DataFrame(columns = ["ds_count", "ds", "date", "value","gain","ratio","total"])


    #This block collects the rate of the benchmarkings along the years and saves it
    #ratio_df = pd.DataFrame(columns = ["ds_count", "ds", "date", "value","gain","ratio","total"])

    #len(sota["result"])
    max(sota["result"])
    min(sota["result"])

    i=0
    for res in sota["result"]:
        if sota.loc[sota.index[i], 'result'] <= max(sota["result"]) :
            if(i == 0):
                gain = sota.loc[sota.index[i], 'result']
            else :
                gain = round(sota.loc[sota.index[i], 'result'] - sota.loc[sota.index[i-1], 'result'],1)

            total = round( gain / max(sota["result"]) ,2)
            #total = round((sota.loc[sota.index[i+1], 'result'] - sota.loc[sota.index[i], 'result'] ),2)
            #print(sota["result"][i+1] - sota["result"][i])
            #OK print(total.astype(str)+ " : " + max(sota["result"]).astype(str) )

            #year=sota.loc[sota.index[i], 'date'].strftime('%Y-%m-%d')
            year=sota.loc[sota.index[i], 'date'].strftime('%Y-%m')
            value=sota.loc[sota.index[i], 'result'].astype(str)

            benchmarking=sota.loc[sota.index[i], 'benchmark_process_class_label']
            #benchmarking = df.replace(to_replace =' model in .*$', value = '', regex = True)
            import re

            #reduce the name of the benchmark
            #benchmarking = re.sub(r'\-.*',"",benchmarking) 
            #re.sub(r'\.\.\..*',"",test)

            new_row = {'ds_count':ds_count, 'ds':benchmarking, 'date':year, 'value':value, 'gain':gain.astype(str), 'ratio':total.astype(str), 'total':max(sota["result"]).astype(str)}
            ratio_df = ratio_df.append(new_row, ignore_index=True)

            i = i+1
    return ratio_df

    

In [6]:
#return ratio_df_all dataframe per desired measure
def get_ratio_df_all_per_measure(desired_measure):

    
    query = """
        PREFIX edam: <http://edamontology.org/>
        PREFIX obo:  <http://purl.obolibrary.org/obo/>
        PREFIX ito:  <https://identifiers.org/ito:>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX foaf: <http://xmlns.com/foaf/0.1/>

        SELECT *
        WHERE {
            ?paper a edam:data_0971 . 
            ?paper rdfs:label ?paperTitle. 
            ?paper obo:date ?date. 

            ?benchmark_process_individual 	rdfs:seeAlso ?paper ;
                                            rdfs:label ?model_label ;
                                            a ?benchmark_process_class . # this will create a place holder for the rdfs:type results that contains the information about the individual

            ?benchmark_process_class rdfs:label ?benchmark_process_class_label ;
                                     rdfs:subClassOf* ito:ITO_00141 . # this limits for the NLP class


            ?performance_measure rdfs:subPropertyOf* ito:performance_measure .
            ?performance_measure rdfs:label ?metricName .



            ?performance_measure rdfs:label ?metricName .
            ?benchmark_process_individual ?performance_measure ?result .	


            #FILTER regex(?paperTitle, "Quaternion Knowledge Graph Embeddings" ). # this searches a specific match
            FILTER regex(?metricName, "^"""+ desired_measure +"""$" ). # this searches a specific match
            #FILTER regex(?benchmark_process_class_label, " - Link Prediction benchmarking$" ). # this searches a specific match
            
            FILTER regex(?benchmark_process_class_label, " - Sentiment Analysis benchmarking" ). # this searches a specific match

            #WN18RR - Link Prediction benchmarking




        } ORDER by ?date



        """

    #send query via API
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    #process results as JSON
    processed_results = json.load(result.response)

    #Use accessory function to process results
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))

            out.append(item)


    #this is the final df containing the results of the query
    df2 = pd.DataFrame(out, columns=cols)

    #now get the unique list of benchmark datasets
    benchmark_process_class_label = df2["benchmark_process_class_label"].unique()

    #start here the ds_count, which is actually the unique combination of 'desired_measure' and 'desired_benchmark'
    ds_count = 1
    #this place holder will contain the plots to be contatenated in the final plot
    plt_data = []

    #create final ratio_df_all for all the combinations
    ratio_df_all = pd.DataFrame(columns = ["ds_count", "ds", "date", "value","gain","ratio","total"])
    #fig = go.Figure()

    #iterate over the benchmark list to create the ratio_df_all, which will be used to plot later
    for desired_benchmark in benchmark_process_class_label:

        #print(desired_benchmark)
        ratio_df = create_ratio(desired_measure,desired_benchmark,ds_count)
        #print("ds_count: ",ds_count)
        #ratio_df = create_ratio(desired_measure,"Citeseer",ds_count)

        if ratio_df is None:
                print("null")  
        elif len(ratio_df.index)>0:
            #print(desired_benchmark)  
            ratio_df_all = ratio_df_all.append(ratio_df, ignore_index=True)
        
        #ds_count = ds_count + 1

    
    if len(ratio_df_all.index) > 0:
        
        print(len(ratio_df_all.index))
        #edit here the column with the benchmark name
        ratio_df_all = ratio_df_all.replace(to_replace =' \- .*', value = '', regex = True)

        #add a column representing the percentual of the max obtained value per value
        ratio_df_all["percent"] = round(ratio_df_all["value"].astype(float)/max(ratio_df_all["total"].astype(float)),2)*100

        #add one extra column wihth the desired measure name in order to condendate the plot
        ratio_df_all["merge"]=desired_measure


        
    return ratio_df_all

In [7]:
pd.DataFrame(metricName)

,0
0,1 in 10 R-at-1
1,Accuracy
2,Average
3,DVD
4,Books
5,Electronics
6,Kitchen
7,Error
8,F1-score
9,F1


In [8]:
len(metricName)

14

In [9]:
#this will concatenate all the get_ratio_df_all_per_measure into only one
get_ratio_df_all_per_global = pd.DataFrame(columns = ["ds_count", "ds", "date", "value","gain","ratio","total"])

for desired_measure in metricName:
    
    #desired_measure = "Accuracy"
    #this is necessary to query the database correctly, it will be editaded later in the code.
    desired_measure = re.escape(desired_measure).replace("\\", "\\\\")
    print(desired_measure)
    
    ratio_df_all_per_measure=get_ratio_df_all_per_measure(desired_measure)
    
    get_ratio_df_all_per_global=get_ratio_df_all_per_global.append(ratio_df_all_per_measure)
    

1\\ in\\ 10\\ R\\-at\\-1
Creating ratio df for  1\\ in\\ 10\\ R\\-at\\-1 ,  1B Words - Sentiment Analysis benchmarking , ds_count= 1


<ipython-input-5-5cb8ad4d6ffd>:82: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.new = df.replace(to_replace =' model in .*$', value = '', regex = True)


1
Accuracy
Creating ratio df for  Accuracy ,  SST-5 Fine-grained classification - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  SST-2 Binary classification - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  IMDb - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  Amazon Review Full - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  Amazon Review Polarity - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  Sogou News - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  MR - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  CR - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  MPQA - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df for  Accuracy ,  Twitter - Sentiment Analysis benchmarking , ds_count= 1
Creating ratio df fo

In [27]:
#change the column to name the facets of the plot
get_ratio_df_all_per_global = get_ratio_df_all_per_global.rename(columns={'merge': 'metricName'})
get_ratio_df_all_per_global['metricName'] = get_ratio_df_all_per_global['metricName'].str.replace('\\','')
get_ratio_df_all_per_global["metricName"].unique()

<ipython-input-27-f87be73d4c46>:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



array(['MSE', 'R^2', 'Average Recall', 'F1-score', 'Error', 'Accuracy',
       'Books', 'DVD', 'Average', 'Electronics', 'Kitchen', 'F1 score',
       '1 in 10 R-at-1', 'F1'], dtype=object)

In [11]:
#check the number of times each measure was obtained for a given ds
#get_ratio_df_all_per_global[get_ratio_df_all_per_global["ds"]=="SQuAD1.1"]


In [28]:
fig = px.density_heatmap(get_ratio_df_all_per_global, x="date", y="percent",
       title="Density heatmaps for Date and Percent of max score based on metricName", 
#      marginal_x="histogram", #too big for image
#      marginal_y="histogram",
       facet_col="metricName", 
       facet_col_wrap= 7, 
       facet_col_spacing=0.0111, 
       facet_row_spacing= 0.02, 
       color_continuous_scale=['#fffdc9', 'red'])

fig.update_layout(
    autosize=False,
    width=1920,
    height=1920)

fig.show()

In [29]:
get_ratio_df_all_per_global


,ds_count,ds,date,value,gain,ratio,total,percent,metricName
0,1,FiQA,2019-08,0.07,0.07,1.0,0.07,100.0,MSE
0,1,FiQA,2019-08,0.55,0.55,1.0,0.55,100.0,R^2
2,1,SemEval 2017 Task 4-A,2019-08,0.61,0.61,1.0,0.61,68.0,Average Recall
0,1,ASTD,2019-08,0.62,0.62,1.0,0.62,69.0,Average Recall
0,1,SemEval,2017-04,0.685,0.685,1.0,0.685,100.0,F1-score
...,...,...,...,...,...,...,...,...,...
14,1,SST-2 Binary classification,2019-06,97.0,1.4,0.01,97.4,100.0,Accuracy
15,1,SST-2 Binary classification,2019-09,97.1,0.1,0.0,97.4,100.0,Accuracy
30,1,Amazon Review Polarity,2019-04,97.37,0.7,0.01,97.37,100.0,Accuracy
16,1,SST-2 Binary classification,2019-10,97.4,0.3,0.0,97.4,100.0,Accuracy


In [30]:
get_ratio_df_all_per_global["ds"].unique()

array(['FiQA', 'SemEval 2017 Task 4-A', 'ASTD', 'SemEval', 'ArSAS',
       'Yelp Fine-grained classification', 'Yelp Binary classification',
       'SST-5 Fine-grained classification', 'Amazon Review Full',
       'Multi-Domain Sentiment Dataset', 'Twitter', 'MR',
       'Financial PhraseBank', 'SST-2 Binary classification', 'MPQA',
       'CR', 'IMDb', '1B Words', 'AJGT', 'DBRD', 'Amazon Review Polarity',
       'ChnSentiCorp', 'ChnSentiCorp Dev', 'HARD', 'Sogou News'],
      dtype=object)

In [31]:
#get_ratio_df_all_per_global[get_ratio_df_all_per_global["ds"]=="ACL-ARC"]

In [32]:
#plot the heatmaps considering the percentual of total value per year for each dataset
fig = px.density_heatmap(get_ratio_df_all_per_global, x="date", y="percent",
       title="Density heatmaps for Date and Percent of max score based on benchmarking",
       facet_col="ds",facet_row="metricName", color_continuous_scale=['#fffdc9', 'red'],
                        facet_col_spacing=0.0050, facet_row_spacing=0.009090)

fig.update_layout(
    autosize=True,
    height=len(get_ratio_df_all_per_global["metricName"].unique())*200,
    width=len(get_ratio_df_all_per_global["metricName"].unique())*300
)

fig.update_yaxes(showticklabels=True, linecolor="black", showgrid=True, gridcolor="black", gridwidth=1 )
fig.update_xaxes(showticklabels=True, linecolor="black", showgrid=True, gridcolor="black", gridwidth=1 )


fig.show()

In [33]:
print(min(get_ratio_df_all_per_global["date"]))
print(max(get_ratio_df_all_per_global["date"]))

2012-06
2020-02


In [34]:
get_ratio_df_all_per_global

,ds_count,ds,date,value,gain,ratio,total,percent,metricName
0,1,FiQA,2019-08,0.07,0.07,1.0,0.07,100.0,MSE
0,1,FiQA,2019-08,0.55,0.55,1.0,0.55,100.0,R^2
2,1,SemEval 2017 Task 4-A,2019-08,0.61,0.61,1.0,0.61,68.0,Average Recall
0,1,ASTD,2019-08,0.62,0.62,1.0,0.62,69.0,Average Recall
0,1,SemEval,2017-04,0.685,0.685,1.0,0.685,100.0,F1-score
...,...,...,...,...,...,...,...,...,...
14,1,SST-2 Binary classification,2019-06,97.0,1.4,0.01,97.4,100.0,Accuracy
15,1,SST-2 Binary classification,2019-09,97.1,0.1,0.0,97.4,100.0,Accuracy
30,1,Amazon Review Polarity,2019-04,97.37,0.7,0.01,97.37,100.0,Accuracy
16,1,SST-2 Binary classification,2019-10,97.4,0.3,0.0,97.4,100.0,Accuracy


In [35]:
#The above graph will display in each cell, how many metrics achieved the performance in 
#in the data range for a given benchmarking (ds). These metrics can be the same or different ones
#which represents two experiments in the save date range for the same dataset.

#for example for MRPC, 2 metrics Accuracy and F1 obtained between 70 and 89 percent of 
#their respective maximum values in this cell in 2013.


get_ratio_df_all_per_global[get_ratio_df_all_per_global["ds"]=="IMDb"]


,ds_count,ds,date,value,gain,ratio,total,percent,metricName
17,1,IMDb,2014-12,92.33,92.33,0.95,97.4,95.0,Accuracy
18,1,IMDb,2016-02,94.1,1.8,0.02,97.4,97.0,Accuracy
19,1,IMDb,2017-12,94.99,0.9,0.01,97.4,98.0,Accuracy
20,1,IMDb,2018-01,95.4,0.4,0.0,97.4,98.0,Accuracy
21,1,IMDb,2019-02,95.68,0.3,0.0,97.4,98.0,Accuracy
22,1,IMDb,2019-04,95.8,0.1,0.0,97.4,98.0,Accuracy
23,1,IMDb,2019-06,96.0,0.2,0.0,97.4,99.0,Accuracy
24,1,IMDb,2019-07,97.4,1.4,0.01,97.4,100.0,Accuracy


In [36]:
get_ratio_df_all_per_global = get_ratio_df_all_per_global.sort_values(by=['value'], ascending=True)

In [38]:
get_ratio_df_all_per_global

,ds_count,ds,date,value,gain,ratio,total,percent,metricName
0,1,FiQA,2019-08,0.07,0.07,1.0,0.07,100.0,MSE
0,1,FiQA,2019-08,0.55,0.55,1.0,0.55,100.0,R^2
2,1,SemEval 2017 Task 4-A,2019-08,0.61,0.61,1.0,0.61,68.0,Average Recall
0,1,ASTD,2019-08,0.62,0.62,1.0,0.62,69.0,Average Recall
0,1,SemEval,2017-04,0.685,0.685,1.0,0.685,100.0,F1-score
...,...,...,...,...,...,...,...,...,...
14,1,SST-2 Binary classification,2019-06,97.0,1.4,0.01,97.4,100.0,Accuracy
15,1,SST-2 Binary classification,2019-09,97.1,0.1,0.0,97.4,100.0,Accuracy
30,1,Amazon Review Polarity,2019-04,97.37,0.7,0.01,97.37,100.0,Accuracy
16,1,SST-2 Binary classification,2019-10,97.4,0.3,0.0,97.4,100.0,Accuracy


In [39]:
#copy the global data frame to facilitate reuse the plots below from the other notebooks
ratio_df = get_ratio_df_all_per_global.copy()

ratio_df['unique_ds']=ratio_df['ds'].astype(str)+', '+ratio_df['metricName']



#ratio_df = ratio_df[ratio_df["metricName"]=="Accuracy"]

ratio_df["percent"]=round((ratio_df["value"].astype(float)/max(ratio_df["percent"].astype(float)))*100,2)

ratio_df = ratio_df.reset_index()

ratio_df["date"]=pd.to_datetime(ratio_df['date'])


#ratio_df["date"]=ratio_df["date"].strftime('%Y-%m')

ratio_df

,index,ds_count,ds,date,value,gain,ratio,total,percent,metricName,unique_ds
0,0,1,FiQA,2019-08-01,0.07,0.07,1.0,0.07,0.07,MSE,"FiQA, MSE"
1,0,1,FiQA,2019-08-01,0.55,0.55,1.0,0.55,0.55,R^2,"FiQA, R^2"
2,2,1,SemEval 2017 Task 4-A,2019-08-01,0.61,0.61,1.0,0.61,0.61,Average Recall,"SemEval 2017 Task 4-A, Average Recall"
3,0,1,ASTD,2019-08-01,0.62,0.62,1.0,0.62,0.62,Average Recall,"ASTD, Average Recall"
4,0,1,SemEval,2017-04-01,0.685,0.685,1.0,0.685,0.68,F1-score,"SemEval, F1-score"
...,...,...,...,...,...,...,...,...,...,...,...
73,14,1,SST-2 Binary classification,2019-06-01,97.0,1.4,0.01,97.4,97.00,Accuracy,"SST-2 Binary classification, Accuracy"
74,15,1,SST-2 Binary classification,2019-09-01,97.1,0.1,0.0,97.4,97.10,Accuracy,"SST-2 Binary classification, Accuracy"
75,30,1,Amazon Review Polarity,2019-04-01,97.37,0.7,0.01,97.37,97.37,Accuracy,"Amazon Review Polarity, Accuracy"
76,16,1,SST-2 Binary classification,2019-10-01,97.4,0.3,0.0,97.4,97.40,Accuracy,"SST-2 Binary classification, Accuracy"


In [40]:
n=0
for i in ratio_df["date"]: 
    i2=i.strftime('%Y-%m')
    ratio_df["date"][n] = i2
    print(ratio_df["date"][n])
    n=n+1

2019-08-01 00:00:00
2019-08-01 00:00:00
2019-08-01 00:00:00
2019-08-01 00:00:00
2017-04-01 00:00:00
2019-08-01 00:00:00
2015-09-01 00:00:00
2015-09-01 00:00:00
2013-10-01 00:00:00
2019-01-01 00:00:00
2014-06-01 00:00:00
2015-02-01 00:00:00
2017-08-01 00:00:00
2018-02-01 00:00:00
2019-10-01 00:00:00
2016-07-01 00:00:00
2017-07-01 00:00:00
2019-04-01 00:00:00
2015-05-01 00:00:00
2015-11-01 00:00:00
2019-02-01 00:00:00
2018-04-01 00:00:00
2015-05-01 00:00:00
2015-05-01 00:00:00
2015-11-01 00:00:00
2015-05-01 00:00:00
2018-04-01 00:00:00
2017-02-01 00:00:00
2015-11-01 00:00:00
2017-02-01 00:00:00
2018-04-01 00:00:00
2015-11-01 00:00:00
2015-05-01 00:00:00
2018-10-01 00:00:00
2018-10-01 00:00:00
2018-04-01 00:00:00
2015-11-01 00:00:00
2018-10-01 00:00:00
2018-02-01 00:00:00
2017-02-01 00:00:00
2019-08-01 00:00:00
2013-10-01 00:00:00
2018-10-01 00:00:00
2019-08-01 00:00:00
2018-05-01 00:00:00
2014-08-01 00:00:00
2018-03-01 00:00:00
2015-06-01 00:00:00
2018-05-01 00:00:00
2016-03-01 00:00:00


<ipython-input-40-f94d0525c35c>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
ratio_df = ratio_df.sort_values(by=['value'], ascending=True)

In [63]:
import pandas as pd
import plotly.express as px

fig1 = px.scatter(ratio_df, x = 'date', y = 'unique_ds', color='unique_ds',  text="value"
              ) 


fig2 = px.line(ratio_df, x = 'date', y = 'unique_ds', color='unique_ds'
              )    

fig2.update_layout(showlegend=False)

fig3 = go.Figure(data=fig1.data + fig2.data)

fig3.update_layout(
    autosize=True,
    width=1040,
    height=1040)


fig3.update_traces(textposition='top center')
